In [2]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml

from utility import *

import warnings
warnings.filterwarnings('ignore')

In [4]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
period = params['periods']
summary_dir = params['summary_dir']

In [6]:
#data inputs 
df_tours = pd.read_parquet(_join(preprocess_dir, 'tour_roster.parquet'))

### Regional Mode Share

In [10]:
#df_region_share = df_tours.groupby(['tour_mode', 'tour_purpose'])['tours'].sum().reset_index()

In [11]:
#regional mode share
# df_region_share['Value'] = df_region_share['tours']/df_region_share['tours'].sum()
# df_region_share = df_region_share.rename(columns={'tour_mode': 'Mode', 'tour_purpose': 'Purpose'})
# df_region_share = df_region_share[['Mode', 'Purpose', 'Value']]

# df_region_share['Concept_ID'] = concept_id
# df_region_share['Metric_ID'] = 'A3.3'
# df_region_share['Metric_name'] = 'Mode Shares'
# df_region_share['Submetric'] = 'A3.3.1'
# df_region_share['Description'] = 'Regional mode share'
# df_region_share['Population'] = 'Whole Population'
# df_region_share['Period'] = ''
# df_region_share['Geography'] = 'Region'
# df_region_share['Zone_ID'] = 'Megaregion'
# df_region_share['Origin_zone'] = ''
# df_region_share['Dest_zone'] = ''
# df_region_share['Dest_zone'] = ''
# df_region_share['Units'] = 'Percentage share'
# df_region_share['Total_Increment'] = 'Increment'

### Regional mode share by period

In [7]:
df_region_period = df_tours.groupby(['Mode', 'Period', 'tour_purpose'])['tours'].sum().reset_index()

df_region_period['Value'] = df_region_period['tours']/df_region_period['tours'].sum()
df_region_period = df_region_period.rename(columns={'tour_purpose': 'Purpose'})
df_region_period = df_region_period[['Mode', 'Period', 'Purpose', 'Value']]

df_region_period['Concept_ID'] = concept_id
df_region_period['Metric_ID'] = 'A3.3'
df_region_period['Metric_name'] = 'Mode Shares'
df_region_period['Submetric'] = 'A3.3.1'
df_region_period['Description'] = 'Regional mode share by time period and tour purpose'
df_region_period['Population'] = 'Whole Population'
df_region_period['Geography'] = 'Region'
df_region_period['Zone_ID'] = 'Megaregion'
df_region_period['Origin_zone'] = ''
df_region_period['Dest_zone'] = ''
df_region_period['Units'] = 'Percentage share'
df_region_period['Total_Increment'] = ''

## Mode Share in Transbay split

In [20]:
df_tb = df_tours.groupby(['transbay_od', 'Mode', 'Period', 'tour_purpose'])['tours'].sum().reset_index()
df_tb = df_tb.loc[df_tb['transbay_od'] == 1]

df_tb['Value'] = df_tb['tours']/df_tb['tours'].sum()
df_tb = df_tb.rename(columns={'tour_purpose': 'Purpose', 'transbay_od' : 'Zone_ID'})
df_tb = df_tb[['Mode', 'Period', 'Purpose', 'Value', 'Zone_ID']]

df_tb['Concept_ID'] = concept_id
df_tb['Metric_ID'] = 'A3.3'
df_tb['Metric_name'] = 'Mode Shares'
df_tb['Submetric'] = 'A3.3.2'
df_tb['Description'] = 'Regional mode share by time period and tour purpose in transbay region'
df_tb['Population'] = 'Whole Population'
df_tb['Geography'] = 'Transbay region'
df_tb['Origin_zone'] = ''
df_tb['Dest_zone'] = ''
df_tb['Units'] = 'Percentage share'
df_tb['Total_Increment'] = ''

### Mode Share by County

In [10]:
df_cnty = df_tours.groupby(['orig_county', 'dest_county', 'Mode', 'Period', 'tour_purpose'])['tours'].sum().reset_index()

df_cnty['Value'] = df_cnty['tours']/df_cnty['tours'].sum()
df_cnty = df_cnty.rename(columns={
                                  'tour_purpose': 'Purpose', 
                                  'orig_county' : 'Origin_zone',
                                  'dest_county' : 'Dest_zone'})
df_cnty = df_cnty[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_cnty['Concept_ID'] = concept_id
df_cnty['Metric_ID'] = 'A3.3'
df_cnty['Metric_name'] = 'Mode Shares'
df_cnty['Submetric'] = 'A3.3.3'
df_cnty['Description'] = 'Regional mode share by time period and tour purpose in origin and destination county'
df_cnty['Population'] = 'Whole Population'
df_cnty['Geography'] = 'County'
df_cnty['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_cnty['Units'] = 'Percentage share'
df_cnty['Total_Increment'] = ''

### Mode Share by RDM Zones

In [11]:
df_rdm = df_tours.groupby(['orig_rdm_zones', 'dest_rdm_zones', 'Mode', 'Period', 'tour_purpose'])['tours'].sum().reset_index()

df_rdm['Value'] = df_rdm['tours']/df_rdm['tours'].sum()
df_rdm = df_rdm.rename(columns={ 
                                'tour_purpose': 'Purpose', 
                                'orig_rdm_zones' : 'Origin_zone',
                                'dest_rdm_zones' : 'Dest_zone'})
df_rdm = df_rdm[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_rdm['Concept_ID'] = concept_id
df_rdm['Metric_ID'] = 'A3.3'
df_rdm['Metric_name'] = 'Mode Shares'
df_rdm['Submetric'] = 'A3.3.4'
df_rdm['Description'] = 'Regional mode share by time period and tour purpose in origin and destination RDM Zones'
df_rdm['Population'] = 'Whole Population'
df_rdm['Geography'] = 'RDM'
df_rdm['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_rdm['Units'] = 'Percentage share'
df_rdm['Total_Increment'] = ''

### Mode Share by Super District

In [17]:
df_sd = df_tours.groupby(['orig_super_dist', 'dest_super_dist', 'tour_mode', 'Period', 'tour_purpose'])['tours'].sum().reset_index()

df_sd['Value'] = df_sd['tours']/df_sd['tours'].sum()

df_sd = df_sd.rename(columns={'tour_mode': 'Mode', 
                              'tour_purpose': 'Purpose', 
                              'orig_super_dist' : 'Origin_zone',
                              'dest_super_dist' : 'Dest_zone'})
df_sd = df_sd[['Origin_zone', 'Dest_zone', 'Mode', 'Period', 'Purpose', 'Value']]

df_sd['Concept_ID'] = concept_id
df_sd['Metric_ID'] = 'A3.3'
df_sd['Metric_name'] = 'Mode Shares'
df_sd['Submetric'] = 'A3.3.5'
df_sd['Description'] = 'Regional mode share by time period and tour purpose in origin and destination super districts'
df_sd['Population'] = 'Whole Population'
df_sd['Geography'] = 'Super district'
df_sd['Zone_ID'] = ''
#df_cnty['Origin_zone'] = ''
#df_cnty['Dest_zone'] = ''
df_sd['Units'] = 'Percentage share'
df_sd['Total_Increment'] = ''

In [13]:
df_tours.columns

Index(['hh_id', 'person_id', 'tour_purpose', 'orig_taz', 'dest_taz',
       'start_hour', 'end_hour', 'tour_mode', 'sampleRate', 'trip_type',
       'tours', 'num_participants', 'transbay_od', 'orig_rdm_zones',
       'orig_super_dist', 'orig_county', 'dest_rdm_zones', 'dest_super_dist',
       'dest_county', 'home_zone', 'pp_share', 'Period', 'Mode', 'pp_tours'],
      dtype='object')

In [15]:
df_tours['pp_tours'] = df_tours['tours'] * df_tours['pp_share']

df_pp = df_tours.groupby(['Mode', 'Period', 'tour_purpose'])['pp_tours'].sum().reset_index()

df_pp['Value'] = df_pp['pp_tours']/df_pp['pp_tours'].sum()

df_pp = df_pp.rename(columns={'tour_purpose': 'Purpose'})
df_pp = df_pp[['Mode', 'Period', 'Purpose', 'Value']]

df_pp['Concept_ID'] = concept_id
df_pp['Metric_ID'] = 'A3.3'
df_pp['Metric_name'] = 'Mode Shares'
df_pp['Submetric'] = 'A3.3.6'
df_pp['Description'] = 'Regional mode shares by time period and tour purpose'
df_pp['Population'] = 'Prioirty population'
df_pp['Geography'] = 'Region'
df_pp['Zone_ID'] = ''
df_pp['Origin_zone'] = ''
df_pp['Dest_zone'] = ''
df_pp['Units'] = 'Percentage share'
df_pp['Total_Increment'] = ''

In [21]:
all_dfs = [df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]

for dfs in all_dfs:
    metric_name = 'mode_shares_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    dfs.to_csv(_join(summary_dir, metric_name + file_name + '.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0])
    
combined_df = pd.concat([df_region_period, df_tb, df_cnty, df_rdm, df_sd, df_pp]).reset_index(drop=True)
combined_df.to_csv(_join(summary_dir, 'mode_shares_' + 'A3.3' + '.csv'), index=None)

707 A3.3.1
466 A3.3.2
20762 A3.3.3
1646306 A3.3.4
143292 A3.3.5
707 A3.3.6


In [31]:
#df_shares = pd.concat([df_region_period, df_cnty, df_rdm, df_sd], ignore_index=False)

In [32]:
#df_shares.to_csv(_join())

In [1]:
#with pd.ExcelWriter(os.path.join(summary_outputs, 'concept-BY15.xlsx'), engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#    df_region_period.to_excel(writer, sheet_name='A3.3.1', startcol=0, index=False)
#    df_tb.to_excel(writer, sheet_name='A3.3.2', startcol=0, index=False)
#    df_cnty.to_excel(writer, sheet_name='A3.3.3', startcol=0, index=False)
#    df_rdm.to_excel(writer, sheet_name='A3.3.4', startcol=0, index=False)
#    df_sd.to_excel(writer, sheet_name='A3.3.5', startcol=0, index=False)

In [ ]:
#tab1 = df_tours.groupby(['tour_mode', 'transbay_od','tour_purpose', 'pp_share'])['flag'].count().reset_index()
#tab2 = df_tours.groupby(['tour_mode', 'start_hour'])['flag'].count().reset_index()
#tab3 = df_tours.groupby(['tour_mode', 'orig_county', 'dest_county'])['flag'].count().reset_index()
#tab4 = df_tours.groupby(['tour_mode', 'orig_super_dist', 'dest_super_dist'])['flag'].count().reset_index()
#tab4 = df_tours.groupby(['tour_mode', 'orig_rdm_zones', 'dest_rdm_zones'])['flag'].count().reset_index()
#tab1